In [2]:
import kivy
from kivy.app import App 
from kivy.uix.label import Label
from kivy.uix.gridlayout import GridLayout
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty
from kivy.uix.floatlayout import FloatLayout
from kivy.graphics import Rectangle
from kivy.graphics import Color
from kivy.graphics import Line
from kivy.lang import Builder
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.core.window import Window
from kivy.properties import StringProperty
from kivy.properties import NumericProperty

Window.size = (800, 800)



class ConnectionWindow(Screen):
    pass

class VariablesWindow (Screen):
    def save_data(self):
        print("Data Saved")
    #pass

class OutputWindow (Screen):
    pass

class WindowManager(ScreenManager):
    load = StringProperty("")
    connection = StringProperty("")

    




kv = Builder.load_file("my.kv")


class MyApp (App):
    def build (self):
        return kv

#Run Program 
if __name__ == "__main__":
    MyApp().run()





Data Saved
Data Saved
Data Saved
Data Saved
